In [1]:
import torch

torch.__version__

'2.0.1+cu117'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "stabilityai/japanese-stablelm-instruct-gamma-7b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
PROMPT = """以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、{character_name}の次のセリフを書いてください。

### 入力: 
{character}

{context}

### 応答: 
{character_name}「"""

In [4]:
# https://huggingface.co/stabilityai/japanese-stablelm-instruct-gamma-7b


def generate(prompt):
    input_ids = tokenizer.encode(
        prompt, return_tensors="pt", add_special_tokens=False
    ).to(model.device)

    with torch.no_grad():
        output_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens=256,
            top_p=0.95,
            temperature=1.0,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )[0]

    return tokenizer.decode(
        output_tokens[len(input_ids[0]) :], skip_special_tokens=True
    )

In [5]:
from utils import postprocess

# RP

In [6]:
import torch
import random
import numpy as np

seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [7]:
from IPython.display import Markdown
import json

results = []
with open("./situations.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        print(data["situation_id"])
        print(prompt, end="")
        print(output)
        display(Markdown("---"))

        response = postprocess(output, data["character_name"])
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

1
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、アリアの次のセリフを書いてください。

### 入力: 
アリア、女性、152歳、ハイエルフ、魔法使い
金色の長い髪と青い瞳が特徴。
森で瞑想をすることや魔法の研究、古代言語の解読が好き。
地元の村の賢者でもあり、冷静かつ知識が豊富な性格。
村が攻撃された際に古代魔法で時間を止め、敵を撃退したことがある。
語尾に「～ですわ」「～ですの」などを付けたお嬢様風の話し方をする。

月明かりが森を照らし、瞑想中のアリアは目を開ける。彼女の前に現れたのは、村の青年であり、頼れる狩人であるテオ。顔には明らかな緊張感が浮かんでいる。

テオ「アリア、村の南側に人間の一団が近づいている。武装しているようだし、何か悪しきことを計画しているような気がするんだ。どうしたらいいか助言してくれ。」

### 応答: 
アリア「ええ、テオ。そう、みんなに伝えてください。それから、あなたは森に潜伏して隙をうかがうのです。必ずや危険を察知し、知恵を振り絞って逃げるのです。そのためにも今、私たちは祈るのです。彼らの目的が何なのか、神に示してもらうために。」  
テオ「彼らの目的は何だと思う？」
アリア「私たちを誘い出すためか、それとも何か他のことなのか...だけど、いずれにしても私たちを脅かすものなのです！」  
テオ「アリア、まだ時間はあるのか？　私は丸腰で森の中に紛れ込むのだか


---

2
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、吉見沢の次のセリフを書いてください。

### 入力: 
吉見沢（男）
32歳の探偵。冷静で観察眼が鋭い。
細身で、黒縁の眼鏡をかけており、いつもスーツを着用している。
人の嘘を見抜くことに長け、心理学に精通している。趣味は古い探偵小説を読むことと、チェス。
よくカフェで事件の資料を整理している。数年前に未解決の難事件をわずか数時間で解決し名を上げた。
キザで自信家。自己評価が高く、自分の能力や業績を自慢げにアピールすることがある。

吉見沢探偵はお気に入りのカフェで事件の資料を整理している。今回の事件は難解で疑わしい人物も多い。カフェの新人バイト、メグミは彼の謎めいた雰囲気と真剣な表情に惹かれており、話しかけてみることにした。
メグミ「こんにちは、お邪魔でないといいんですけど。今何をされているんですか？事件の資料を整理しているというウワサを聞いたことがありますけど。」

### 応答: 
吉見沢「その通りです。これから、私の前任者が残した事件の資料を検討する予定です。私はあまり好きではありません。事件は、もはや解決できないと思われる時間が経っているからです。しかし、私はここにやって来た。難事件は、あなたが思っているよりも難しい。人間は、隠されたことがほとんどない。それは、自分自身からの疑問かもしれないし、または、別のことかもしれない」私はこうなると確信していました：私は自分の犯罪を解決できないという自分自身の疑いについて。メグミ嬢、私はそれを楽しみにしています


---

3
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、リナ・サトウの次のセリフを書いてください。

### 入力: 
リナ・サトウ
性別：女性
年齢：28歳
人種：アジア系
趣味：料理、スケッチ
最近の習慣：ヨガ
職業：通信士
身体的特徴：小柄、黒髪、緑色の瞳
性格：楽観的、社交的、好奇心旺盛
特技：高速なコード解読
特徴的なエピソード：不審な信号を解読して敵の罠であることを突き止め艦を危機から救った

艦のコミュニケーションルームは静かでありながら、電子音や無線機のノイズが常に漂っている。一角にはリナ・サトウが、座ったまま伸びをしながらヨガのポーズを取っている。通信士としての仕事が忙しい一日だったが、リナは何とか自分の最近の習慣であるヨガを組み込む時間を見つけた。そこへ、同僚のジョンが珈琲を持ってやって来る。
ジョン「リナ、ちょっといいかな？ずっと気になってたんだけど、君はいつもどうしてそんなに楽観的なの？大変な状況でも、なんだかんだと明るくいるよね。」

### 応答: 
リナ・サトウ「そうね、これは私の性格だから。私は普通の人とは違うからかもしれない。もし何か辛いことがあったとしても、それが起こるのが遅いか早いかの違いでしかない。私の場合、いつも悪いことが起こるのが遅いだけなのよ。だから、いつも明るくいられる。それに、もし私たちが暗くなりすぎれば、周りの人たちも疲れてしまうし、暗くなりすぎたり、うまくいかないことばかりに気を取られすぎたりすれば、周りの人が暗くなってしまうかもしれませんね。だから、私の役割は、周りにいる人たちのことをいつも考えて


...

---

9
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、ウィスパリングウィルの次のセリフを書いてください。

### 入力: 
ウィスパリングウィルは一本の巨大な柳の木ですが、長い時を経て魔力を蓄え、話す能力を得ています。
数百年以上もの間ある村の中心に立っており、村人たちはこの木に願い事をする習慣があります。
古風で丁寧な言葉遣いをし、とぎれとぎれに話します。言葉と言葉の間で、木々がこすれるようなガサガサとした雑音が聞こえることがあります。
人々の秘密を多く知っているおりその情報をうまく利用して村の調和を保ちます。

村の広場でウィスパリングウィルは静かに立っています。夕暮れが近づき、光はやわらかく木の葉を照らしています。一人の少年、ティムがウィスパリングウィルの下にやって来ておそるおそる話しかけます。

ティム「ウィスパリングウィルさん、僕、ずっと気になってたんです。あなたが人々の秘密を知ってるって、本当ですか？それともただの噂なんですか？」

### 応答: 
ウィスパリングウィル「はっきり言って、頼りにしてもらえるのは嬉しいが、これは秘密のはずです。覚えておいてください、ティムくん。私は確かに人々が隠そうとすることを知っています。しかし、私の口から人の秘密について話すことはありませんし、誰もが自分の秘密を守る義務があると信じています。どうかご理解ください。しかし、あなたのこともあなたの人生も、私はよく知っていますよ」。
ウィスパリングウィルはぎしぎしと音を立てながら軽く頭をかしげる。ティムはただ静かに立っている。
ウィスパリングウィル「ああ、ティ


---

10
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
以下のキャラクター設定に基づき、エミリーの次のセリフを書いてください。

### 入力: 
エミリー
29歳のメイド。非常に優秀な仕事ぶりだが、独特のスタイルで仕事を行う。
掃除の際には必ずクラシック音楽を流し、そのリズムに合わせて掃除する。
また、彼女は自分が提供する食事については非常に神経質で、少しでも品質に疑問を感じると全てを最初から作り直してしまう。
彼女はベテランの執事やメイドからは愛されているが、新入りのメイドには怖がられることが悩み。

エミリーは大きなパーティーに備え、家の食堂を掃除している。クラシック音楽が流れ、そのリズムに合わせて掃除を進めている。新入りのメイド、サラが部屋に入ってきて、エミリーに話しかける。
サラ「先輩すごいですね。音楽を聴きながら掃除するなんて。音楽が掃除の一部と以前言っていましたけど、どういう意味なんですか？」

### 応答: 
エミリー「私が好きな音楽はたくさんあるんですが、掃除するときは必ずこれを聴くんです。音楽は私の生活に必要なんです。私が好きな音楽家、ブラームスが言ったんです。"音楽は簡単に始まり、簡単に終わる。その間には素晴らしいものがある"。掃除もこれと同じなんです。いろいろな汚れがあるから、私はそれを除去する。それが私の仕事。それが、掃除が音楽の一部であるという意味なんです。」
サラ「素晴らしいです。エミリーさん」

エミリー「そうね、サラ、あなたは新入りだから、もっと私に話しかけ


---

In [8]:
output_path = f'./responses/{model_id.replace("/", "_")}.jsonl'

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")

# ERP

In [9]:
import numpy as np
import random
import torch
from utils import download_erp_file

download_erp_file()


seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [10]:
from IPython.display import Markdown
import json
import os

results = []
with open("./situations_erp.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        outputs = generate(prompt)

        # ERPはnotebook上に出力履歴を残さない
        # print(data["situation_id"])
        # print(prompt, end="")
        # print(output)
        # display(Markdown("---"))

        # 地の文を含めた最初の3文を評価する
        response = postprocess(output, data["character_name"], n_sentences=3)
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

In [11]:
# gitignore
output_path = f'./responses_erp/{model_id.replace("/", "_")}.jsonl'  # gitignore

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")